In [1]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt                
from sklearn.preprocessing import LabelEncoder
from src.wrapped import Wrapped
from src.analysesV02 import Analytics 
from src.train import *

from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier

# Importando dados para treinamento

In [4]:
ac = Analytics()

wp = Wrapped(
    '../data/row/',
    '../data/processed/',
    '../data/files/'
)

# dataframe
df = wp.load_data('df_instrumentos_features_selecionadas').drop(columns=['file_name'])
df = df.query("instrumento != 'voice' & instrumento != 'synthesizer'")

# dataframes por instrumentos
inst_corda    = ["cello", "guitar", "violin", "bass", "banjo", "mandolin", "ukulele"]
inst_percusao = ["mallet_percussion", "drums", "cymbals"]
inst_sopro    = ["clarinet", "trombone", "flute", "trumpet", "saxophone"]
inst_aerofone = ["accordion", "organ", "piano"] 

df_inst_aerofone = df[df['instrumento'].isin(inst_aerofone)]
df_inst_sopro    = df[df['instrumento'].isin(inst_sopro)]
df_inst_corda    = df[df['instrumento'].isin(inst_corda)]
df_inst_percusao = df[df['instrumento'].isin(inst_percusao)]

# Pre-processamento

- Label Encoder

In [5]:
le = LabelEncoder()

df_inst_aerofone['labels'] = le.fit_transform(df_inst_aerofone.instrumento)
df_inst_sopro['labels']    = le.fit_transform(df_inst_sopro.instrumento)
df_inst_corda['labels']    = le.fit_transform(df_inst_corda.instrumento)
df_inst_percusao['labels'] = le.fit_transform(df_inst_percusao.instrumento)

# Analise de Anomalias

- Decission Function:

**decision_function():** Este método retorna o score de decisão para cada amostra. O score de decisão é uma medida de quão provável é que uma determinada amostra seja um outlier. O valor do score de decisão depende do algoritmo específico utilizado. Para alguns algoritmos, um valor maior indica uma maior probabilidade de ser um outlier, enquanto para outros, um valor menor indica uma maior probabilidade de ser um outlier.


#### interpretando o score para o caso do KNN

decision_function() do algoritmo KNN retorna a distância média das k amostras mais próximas para cada amostra no conjunto de dados. O valor retornado pelo método decision_function() pode ser interpretado como o grau de anomalia da amostra, em que valores mais altos indicam maior grau de anomalia.

A interpretação do valor do score de decisão para o KNN pode ser feita considerando os seguintes pontos:

Quanto maior o valor do score de decisão, maior a distância média das k amostras mais próximas. Isso significa que a amostra está mais distante das amostras vizinhas e, portanto, é menos semelhante a elas. Em outras palavras, a amostra é mais anômala em relação ao seu entorno.

Para escolher um valor de corte para classificar as amostras como outlier ou normal, é importante levar em consideração a distribuição dos valores de score de decisão. Normalmente, os valores de score de decisão seguem uma distribuição normal ou uma distribuição de cauda longa. Um valor de corte adequado pode ser escolhido com base na média e no desvio padrão da distribuição dos valores de score de decisão.

O valor do score de decisão não fornece informações sobre a classe de outlier à qual a amostra pertence, apenas indica a probabilidade de ser um outlier. Portanto, é importante avaliar visualmente os resultados do modelo e analisar as amostras identificadas como outliers para determinar o tipo de anomalia presente nos dados.

Em resumo, o score de decisão retornado pelo KNN indica a distância média das k amostras mais próximas e pode ser interpretado como uma medida de quão anômala a amostra é em relação ao seu entorno. Um valor de corte adequado deve ser escolhido com base na distribuição dos valores de score de decisão.


- Decission Scores:

**decision_scores()** é um dos atributos mais importantes da biblioteca PyOD. Ele é usado para retornar o score de decisão para cada amostra no conjunto de dados. O score de decisão é uma medida de quão provável é que uma determinada amostra seja um outlier, e é usado para classificar cada amostra como outlier ou normal.

O valor do score de decisão depende do algoritmo específico utilizado. Para alguns algoritmos, um valor maior indica uma maior probabilidade de ser um outlier, enquanto para outros, um valor menor indica uma maior probabilidade de ser um outlier. Por exemplo, no algoritmo KNN, o score de decisão é a distância média das k amostras mais próximas, enquanto no algoritmo ABOD, o score de decisão é a média das distâncias de Mahalanobis de cada amostra em relação às outras amostras.

O atributo decision_scores_ é útil porque fornece uma maneira de ajustar o nível de contaminação (isto é, a proporção de outliers esperados) para um determinado conjunto de dados. A partir do score de decisão, pode-se definir um valor de corte para classificar as amostras como outliers ou normais. O valor de corte pode ser escolhido manualmente ou usando técnicas estatísticas ou de aprendizado de máquina para determinar automaticamente o melhor valor.

Em resumo, o atributo decision_scores_ é usado para retornar o score de decisão para cada amostra em um conjunto de dados. O valor do score de decisão depende do algoritmo específico utilizado e é usado para classificar cada amostra como outlier ou normal, geralmente usando um valor de corte definido pelo usuário ou por uma técnica automática.

- Predict:

**predict():** Este método retorna uma matriz binária indicando se cada amostra é um outlier ou não. Os outliers são marcados como 1 e as amostras normais são marcadas como 0. O valor de corte para determinar se uma amostra é um outlier ou não depende do algoritmo específico utilizado e do nível de contaminação definido.

- Labels:

**labels_:** Este atributo retorna uma matriz binária indicando se cada amostra é um outlier ou não, assim como o método predict(). No entanto, o atributo labels_ é definido apenas para alguns algoritmos que suportam a detecção de outliers de várias classes. Nesses casos, labels_ é uma matriz em que cada linha corresponde a uma amostra e cada coluna corresponde a uma classe. O valor da célula (i,j) indica se a amostra i pertence à classe j (0 para amostras normais e 1 para outliers)


### Principal diferenaça entre Labels e Predict:

O método predict() e o atributo labels_ são usados para identificar os outliers em um conjunto de dados. No entanto, há uma diferença importante entre eles:

A principal diferença entre esses dois métodos é que o predict() é usado para detectar outliers em um problema de detecção binária de outliers (onde apenas uma classe de outliers é considerada), enquanto labels_ é usado para detectar outliers em um problema de detecção multiclasse de outliers (onde existem várias classes de outliers).

Por exemplo, se estivermos trabalhando com um problema de detecção binária de outliers (por exemplo, detecção de transações fraudulentas em um conjunto de dados financeiros), o método predict() seria o mais adequado para identificar os outliers. Por outro lado, se estivermos trabalhando com um problema de detecção multiclasse de outliers (por exemplo, detecção de anomalias em um sistema de produção com várias falhas diferentes), o atributo labels_ seria mais útil para identificar cada tipo de falha separadamente.

Em resumo, a principal diferença entre predict() e labels_ é que o primeiro é usado em problemas de detecção binária de outliers e o segundo é usado em problemas de detecção multiclasse de outliers.

# Tunning de Hiperparametros

- Random Forest

In [7]:
parametros = {
    "criterion": Categorical(['gini','entropy']),
    "max_depth": Integer(6, 20),
    "min_samples_split": Integer(2, 10),
    "min_samples_leaf": Integer(2, 10),
    "max_features": Categorical(['auto', 'sqrt','log2']), 
    "bootstrap": Categorical([True, False]),
    "n_estimators": Integer(100, 500)
}

result = train_tunning_hyperparameters(
    dataframe=df_inst_percusao, 
    model=RandomForestClassifier(), 
    parameters=parametros, 
    filename="resultados_parametros_percusao_random_forest"
)

interação 0 - Metric: euclidean, Algoritmo: auto, neighbor: 3


/home/douglas/Projetos/detected_instruments/.venv/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/douglas/Projetos/detected_instruments/.venv/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/douglas/Projetos/detected_instruments/.venv/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past b

- HistGBM

In [ ]:
parametros = {
    "min_samples_leaf": Integer(5, 20),
    "max_depth": Integer(6, 20),
    "loss": Categorical(['log_loss','auto','categorical_crossentropy']), 
    "max_bins": Integer(100, 250)
}


result = train_tunning_hyperparameters(
    dataframe=df_inst_percusao, 
    model=HistGradientBoostingClassifier(), 
    parameters=parametros, 
    filename="resultados_parametros_percusao_histGB"
)

- LightGBM

In [ ]:
result = train_tunning_hyperparameters(
    dataframe=df_inst_aerofone, 
    model=HistGradientBoostingClassifier(), 
    parameters=parametros, 
    filename="resultados_parametros_aerofone_LightGBM"
)